## Analyzing the Effects of Space Flight on Telomere Length 
## Dynamics in NASA Astronauts

Space remains a great frontier for humanity, with many inherent dangers to human health. As part of my Ph.D, I'm researching how spaceflight impacts human health. Specifically, I examine time aboard the International Space Station affects telomeres, the structural ends of human DNA, in the blood of NASA's astronauts. My research takes the *first look at the changes to telomeres in unrelated astronauts as a result of spaceflight*, informing NASA policies on future manned missions.

This repository details the context of my research in the wider pursuit of space exploration; a brief overview of my laboratory methods; and an extensive walkthrough of my analysis using Python on telomere length data in astronauts. This repository may serve as an accessory to our upcoming publications. The walkthrough below is intended for those more interested in the science and less in the code. For those more interested in the code as well, please see the Jupyter Notebook file above. 

**Contact:**  
Jared Luxton  
jLuxton@colostate.edu

<p align="center">
<a href="url">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/200px-Python-logo-notext.svg.png" height="150"> 
<img src="https://cdn1.medicalnewstoday.com/content/images/articles/319/319971/space-explorer.jpg" height="150">
<img src="https://abm-website-assets.s3.amazonaws.com/rdmag.com/s3fs-public/embedded_image/2017/04/telomere-chromosome-stock.jpg" height="150">
</a>
</p>
&nbsp;
&nbsp;   

## Table of Contents:
* [Background: Health Risks and Obstacles to Space Flight](#background) 
* [Approach: Identifying Risks by Measuring Telomere Lengths](#approach)
* [Methods: Blood Collections, Cell Culture, Telomere Measurements](#methods)
* [Data Cleaning: Handling Telomere Length Data](#data-cleaning)
* [Data Analysis: Visualization and Statistics](#data-analysis)
* [Conclusions](#conclusions)

&nbsp;    

## Background 
**Health Risks and Obstacles to Space Flight**\
In recent decades, the possibilities of extended, manned explorations of space, permanent lunar bases, and even human colonies on other planets in our solar system, i.e Mars, have become more technologically feasible and amazingly, close to realization. However, extended time spent living in space, on potential lunar bases, or on Mars, comes with specific dangers and risks to human health that are not found on Earth: space radiation and microgravity. My research concerns the former. Space radiation refers to two specific types of ionizing radiation encountered in space: solar particle emissions (SPEs) and galactic cosmic rays (GCRs). We don't experience either of these on Earth thanks to shielding by the Earth's magnetosphere. 

SPEs occur when Sun emits waves of high energy particles, mainly protons, while GCRs are extremely high energy atomic nuclei, hurtling through space at nearly the speed of light. When either come in contact with human cells they wreak havoc, ripping off electrons and causing cellular and DNA damage. This can lead to cancer and/or tissue degeneration. With current technology, shielding spacecraft agaisnt space radiation is unfeasable due to cost and payload weight. Thus, we must research the consequences of exposure to best address them. As well, simulating space radiation on Earth is impossible for cost and technology reaosns; we therefore must study the effects of space radiation on the biology of astronauts aboard the International Space Station. But what aspects of human biology should we focus on? In our lab we focus on telomeres.

Telomeres are repetitive sequences of DNA encapsulated by protein and RNA found at the ends of chromosomes.

&nbsp; 

## Approach 
**Identifying Risks by Measuring Telomere Lengths**\
...

&nbsp; 

## Methods
**Blood Collections, Cell Culture, Telomere Measurements**\
...

&nbsp; 

## Data Cleaning 
**Handling Telomere Length Data**\
...

&nbsp; 

## Data Analysis
**Visualization and Statistics**\
...

&nbsp; 

## Conclusions
**Highlights and Final Thoughts**\
...

&nbsp; 
&nbsp; 